파이썬의 컬렉션은 모두 반복형이며, 다음과 같은 연산을 지원하기 위해 내부적으로 반복자를 사용한다. 

- for 루프
- 컬렉션형 생성과 확장
- 텍스트 파일을 한 줄씩 반복
- 지능형 리스트/딕셔너리/집합
- 튜플 언패킹
- 함수 호출 시 *를 사용해서 실제 매개변수를 언패킹

이 장에서 다루는 내용  

- 반복형 객체를 처리하기 위해 내부적으로 iter() 내장 함수를 사용하는 방법
- 파이썬에서 고전적인 반복자 패턴을 구현하는 방법
- 제너레이터가 작동하는 방식을 한 줄씩 상세히 설명
- 고전적인 반복자를 제너레이터 함수나 제너레이터 표현식으로 바꾸는 방법
- 표준 라이브러리에서의 범용 제너레이터 함수의 활용
- 제너레이터를 결합하기 위해 새로 추가된 yield from을 사용하는 방법 
- 사례 연구 : 대형 데이터셋을 사용하도록 설계된 데이터베이스 변환 유틸리티에서 제너레이터 함수 이용 
- 비슷해 보이는 제너레이터와 코루틴이 실제로는 아주 다르며, 혼합해서 사용하면 안되는 이유

# 1. Sentence 버전 #1: 단어 시퀀스

In [3]:
import re
import reprlib
 
RE_WORD = re.compile('\w+')
 
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __getitem__(self, index):
        return self.words[index]
    

    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [5]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [6]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [7]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [8]:
s[0]

'The'

### 1.1 Sequence가 반복 가능한 이유 : iter() 함수

파이썬 인터프리터가 x 객체를 반복해야 할 때는 언제나 iter(x)를 자동으로 호출합니다.

여기서 iter() 내장 함수는 다음 과정을 수행합니다.

1. 객체가 __iter __() 메소드를 구현하는지 확인하고, 이 메소드를 호출해서 이터레이터를 가져온다.
2. __iter __() 메소드가 구현되어 있지 않지만 __getitem __()이 구현되어 있다면, 파이썬은 인덱스 0에서 시작해서 항목을 순서대로 가져오는 이터레이터를 생성한다.
3. 이 과정이 모두 실패하면 파이썬은 'TypeError: 'C' object is not iterable'이라는 메세지와 함께 TypeError가 발생한다. 여기서 C는 대상 객체의 클래스이다.


이러한 이유로 모든 파이썬 시퀀스는 반복할 수 있습니다. 시퀀스가 __getitem __()을 구현하고 있기 때문입니다. 사실 표준 시퀀스는 __iter __() 메소드도 구현하고 있으므로 우리가 정의한 시퀀스도 이 메소드를 구현해야 합니다. __getitem __()을 특별히 다루는 이유는 하위 버전과의 호환성을 유지하기 위함이며, 미래에는 사라질 수도 있습니다.

 

이처럼 __iter __() 스페셜 메소드를 구현하는 객체뿐만 아니라 0에서 시작하는 정수형 키를 받는 __getitem __() 메소드를 구현하는 객체도 반복형으로 간주하는 것은 덕 타이핑의 극단적인 형태입니다.

# 2. 반복형과 반복자

Iterable (반복형) : iter() 내장 함수가 이터레이터(iterator, 반복자)를 가져올 수 있는 모든 객체와 이터레이터를 반환하는 __iter __() 메소드를 구현하는 객체는 반복형이다. 0에서 시작하는 인덱스를 받는 __getitem __() 메소드를 구현하는 객체인 시퀀스도 마찬가지다.
Iterable과 Iterator의 관계를 명확히 구분하는 것은 중요합니다. 파이썬은 반복형 객체에서 이터레이터를 가져옵니다.

 

다음 코드는 문자열을 반복하는 간단한 for 루프입니다. 여기서 'ABC' 문자열은 반복형입니다. 이터레이터가 보이지 않지만, 내부 어딘가에 있습니다. 

In [10]:
#for 문 
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [25]:
#while 문
s = 'ABC'
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


이터레이터에 대한 표준 인터페이스는 다음과 같은 메소드 두 개를 정의합니다.

- __next __() : 다음에 사용할 항목을 반환한다. 더 이상 항목이 남아 있지 않으면 StopIteration을 발생시킨다.
- __iter __() : self를 반환한다. 그러면 for 루프 등 반복형이 필요한 곳에 이터레이터를 사용할 수 있게 해준다.

In [27]:
s3 = Sentence("Pig and Pepper")
it = iter(s3)
it
next(it)

'Pig'

In [28]:
next(it)

'and'

In [29]:
next(it)

'Pepper'

In [30]:
next(it)

StopIteration: 

In [31]:
list(it)

[]

In [32]:
list(s3)

['Pig', 'and', 'Pepper']

이터레이터가 필수적으로 구현해야 하는 메소드는 __next __()와 __iter __() 밖에 없으므로, next()를 호출하고 StopIteration 예외를 잡는 방법 외에는 항목이 소진되었는지 확인할 방법이 없습니다. 그리고 이터레이터는 'reset'할 수 없습니다. 다시 처음부터 반복해야 한다면 처음 이터레이터를 생성했던 반복형에 iter()를 호출해야 합니다. 이터레이터 자체에 iter()를 호출하는 것은 소용이 없습니다. 앞에서 설명한 것처럼 Iterator.__iter __()는 단지 self를 반환하도록 구현되어 있으므로 소진된 이터레이터는 재설정하지 못합니다.

 

따라서 이터레이터는 다음과 같이 정의내릴 수 있습니다.

 - Iterator(반복자) : 다음 항목을 반환하거나, 다음 항목이 없을 때 StopIteration 예외를 발생시키는, 인수를 받지 않는 __next __() 메소드를 구현하는 객체. 파이썬의 이터레이터는 __iter __() 메소드도 구현하므로 반복형이기도 하다.

내장 함수 iter()가 시퀀스에 제공하는 특별한 처리 덕분에 위에서 구현한 Sentence 클래스는 반복형이었습니다. 이제 표준 반복형 프로토콜을 구현해보도록 하겠습니다.

# 3. Sentence 버전 #2 : 고전적인 반복자

Sentence 클래스의 다음 버전은 고전적인 이터레이터 디자인 패턴에 맞춰서 구현합니다. 뒤에 리팩토링하면서 명확해지지만, 이 코드는 파이썬의 관용적인 방법은 아닙니다. 하지만 반복형 컬렉션과 이터레이터 객체 간의 관계를 명확히 정의하는데 큰 도움이 됩니다.

 

아래 구현된 Sentence 클래스가 반복형입니다. __iter __() 스페셜 메소드를 구현하고 있고, 이 메소드가 SentenceIterator를 반환하기 때문입니다. 이 방식이 원래 '디자인 패턴(GoF book)'에서 설명하고 있는 이터레이터 디자인 패턴입니다.

 

여기서는 반복형과 이터레이터의 차이 및 이 둘이 어떻게 연관되는지 명확히 보여주기 위해서 다음고 같이 구현되었습니다.

In [39]:
# 반복자 패턴을 이용한 Setence 구현 

import re
import reprlib
 
RE_WORD = re.compile('\w+')
 
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
 
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    def __iter__(self):
        return SentenceIterator(self.words)

class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
    
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word 
    
    def __iter__(self):
        return self

Sentence가 통과한 테스트를 모두 통과합니다. 단, __getitem __() 메소드는 구현되어 있지 않기 때문에 인덱스로 접근할 수는 없습니다.

In [46]:
s3 = Sentence('"The time has come," the Walrus said,')

In [47]:
list(s3)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [48]:
s3[0]

TypeError: 'Sentence' object is not subscriptable

### 3.1 Sentence를 반복자로 만들기 : 좋지 않은 생각  

  
  반복형과 이터레이터를 만드는 데 있어서 흔히 발생하는 오류는 둘을 혼동하기 때문에 발생합니다. 간단히 정리하면, 반복형은 호출될 때마다 반복자를 새로 생성하는 __iter __() 메소드를 가지고 있습니다. 이터레이터는 개별 항목을 반환하는 __next __() 메소드와 self를 반환하는 __iter __() 메소드를 가지고 있습니다.

 

따라서 이터레이터는 반복형이지만, 반복형은 이터레이터가 아닙니다.

 

Sentence 클래스 안에 __iter __() 외에 __next __()도 구현해서 Sentence 객체를 반복형이나 이터레이터로 만들고 싶을 수도 있습니다. 그러나 이는 정말 잘못된 생각입니다.

 

'디자인 패턴'의 이터레이터 디자인 패턴 중 'Applicability' 섹션에서는 다음과 같은 용도로 이터레이터 패턴을 사용하라도 설명하고 있습니다.

반복자 패턴은 다음과 같은 용도에 사용하라.  

 - 집합 객체의 내부 표현을 노출시키지 않고 내용에 접근하는 경우
 - 집합 객체의 다중 반복을 지원하는 경우
 - 다양한 집합 구조체를 반복하기 위한 통일된 인터페이스를 제공하는 경우
  
  
다중 반복을 지원하려면 동일한 반복형 객체로부터 여러 독립적인 이터레이터를 가질 수 있어야 하며, 각 이터레이터는 고유한 내부 상태를 유지해야 합니다. 따라서 이 패턴을 제대로 구현하려면 iter(my_iterable)을 호출할 때마다 독립적인 이터레이터가 새로 만들어져야 합니다. 그렇기 때문에 두 번째 Sentence 클래스에서 SentenceIterator가 필요한 것입니다.

# 4. Sentence 버전 #3 : 제너레이터 함수

In [50]:
# 제너레이터 함수를 사용해서 구현한 Sentence

import re
import reprlib

RE_WORD = re.compile('\w+')
 
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
 
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words: 
            yield word 
        return # 함수가 끝에 도달하면 값을 자동으로 반환하므로, 이 return은 사실 필요없다
    

두 번째 버전에서 Sentence 클래스의 __iter __()는 SentenceIterator() 생성자를 호출해서 이터레이터를 생성하고 반환했습니다. 세 번째 버전의 이터레이터는 사실 제너레이터 객체로서, __iter __() 메소드를 호출할 때 자동으로 생성됩니다. 여기서 __iter __()는 제너레이터 함수이기 때문입니다.

In [51]:
s = Sentence('"The time has come," the Walrus said,')

for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [52]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

### 4.1 제너레이터 함수의 작동 방식

함수 바디 안에 yield 키워드를 가진 함수는 모두 제너레이터 함수입니다. 제너레이터 함수는 호출되면 제너레이터 객체를 반환합니다. 즉, 제너레이터 함수는 제너레이터 팩토리라고 할 수 있습니다.

In [53]:
def gen_123():
    yield 1
    yield 2
    yield 3

In [57]:
gen_123 # doctest: +ELLIPSIS

<function __main__.gen_123()>

In [60]:
gen_123() # doctest: +ELLIPSIS

<generator object gen_123 at 0x0000021C46F445F0>

In [61]:
for i in gen_123(): 
    print(i)

1
2
3


In [62]:
g = gen_123()
next(g)

1

In [63]:
next(g)

2

In [64]:
next(g)

3

In [65]:
next(g)

StopIteration: 

제너레이터 함수는 함수 바디를 포함하는 제너레이터 객체를 생성합니다. next()를 제너레이터 객체에 호출하면 함수 본체에 있는 다음 yield로 진행하며, next()는 함수 바디가 중단된 곳에서 생성된 값을 평가합니다. 마지막으로 함수 바디가 반환될 때 이 함수를 포함하고 있는 제너레이터 객체는 Iterator 프로토콜에 따라 StopIteration 예외를 발생시킵니다.  
  
제너레이터에서 가져온 결과에 대해 이야기할 때는 조금 더 명확히 하는 것이 좋은데, 제너레이터는 값을 생성합니다. 하지만 제너레이터가 값을 '반환'한다고 하면 조금 혼란스럽습니다. 함수는 값을 반환합니다. 제너레이터 함수를 호출하면 제너레이터 객체가 반환됩니다. 그리고 제너레이터 객체는 값을 생성합니다. 제너레이터 객체는 일반적인 방식으로 값을 '반환'하지 않습니다. 제너레이터 함수 안에 있는 return 문은 제너레이터 객체가 StopIteration 예외를 발생하게 만듭니다.  
 
다음 예제 코드는 for 루프와 함수 바디 간의 상호작용을 좀 더 명확히 보여줍니다.

In [66]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')
    
for c in gen_AB():
    print(' -->', c)

start
 --> A
continue
 --> B
end.


이제 세 번째 버전의 Sentence.__iter __()가 어떻게 동작하는지 조금 명확해졌을 것이라고 생각됩니다. __iter __()는 제너레이터 함수로서, 호출되면 이터레이터 인터페이스를 구현하는 제너레이터 객체를 생성합니다. 따라서 SentenceIterator 클래스가 더 이상 필요하지 않습니다.

 

세 번째 버전의 Sentence 버전의 이전 버전들보다 훨씬 짧지만, 데이터들을 지연시켜 생성하는 것은 아닙니다. 따라서 데이터를 지연시키도록 구현할 수 있습니다. 이러한 지연은 적어도 프로그래밍 언어와 API에서만큼은 좋은 성질이라고 여겨지며, 이렇게 지연되는 것은 가능한 한 최후의 순간까지 값 생산을 연기합니다. 이렇게 함으로써 메모리를 줄일 수 있을 뿐만 아니라 불필요한 처리도 피할 수 있습니다.

# 5. 느긋한 구현

느긋한 구현 : 한 번에 한 항목만 생성  < == > 조급한 구현


# 6. Sentence 버전 #5 : 제너레이터 표현식

위에서 구현한 Sentence 클래스의 간단한 제너레이터 함수는 제너레이터 표현식으로 바꿀 수 있습니다.

 

제너레이터 표현식은 리스트 컴프리헨션의 lazy 버전이라고 생각할 수 있습니다. 처음부터 전체 리스트를 생성하는 대신, 필요에 따라 항목을 그때그때 생성하는 제너레이터를 반환하기 때문입니다. 즉, 리스트 컴프리헨션이 리스트 팩토리라면 제너레이터 표현식은 제너레이터 팩토리라고 생각할 수 있습니다.

 

다음 코드는 제너레이터 표현식을 리스트 컴프리헨션과 간단히 비교합니다.

In [71]:
res1 = [x*3 for x in gen_AB()]

start
continue
end.


In [74]:
for i in res1:
    print(i)

AAA
BBB


In [76]:
res2 = (x*3 for x in gen_AB())
res2

<generator object <genexpr> at 0x0000021C46F45460>

In [77]:
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


결국 제너레이터 표현식은 제너레이터를 생성하고, 제너레이터 표현식을 사용하면 Sentence 클래스의 코드를 더 짧게 만들 수 있습니다.

In [78]:
import re
import reprlib
 
RE_WORD = re.compile('\w+')
 
class Sentence:
    def __init__(self, text):
        self.text = text
 
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))


네 번째 버전과 __iter __() 메소드만 다릅니다. 여기서는 제너레이터 함수가 아니며(yield 키워드가 없음), 제너레이터를 생성해서 반환하는 제너레이터 표현식을 사용합니다. 실행 결과는 네 번째 버전과 마찬가지로 __iter __() 메소드의 호출자가 제너레이터 객체를 받습니다.

 

제너레이터 표현식은 편리 구문(syntactic sugar)으로서, 제너레이터 함수를 대체할 수 있지만 더 편리한 경우도 종종 있습니다. 아래에서는 제너레이터 표현식을 사용하는 방법에 대해 알아보겠습니다.

# 7. 제너레이터 표현식: 언제 사용하나?

Vector 클래스를 구현할 때 제너레이터 표현식을 여러 번 사용했습니다. __eq __(), __hash __(), __abs __(), angle(), angles(), format(), __add __(), __mul __() 메소드가 각각 제너레이터 표현식을 사용합니다. 이들 메소드에서 리스트 컴프리헨션을 사용해도 제대로 작동하겠지만, 그러면 중간 리스트 값들을 저장하기 위해 메모리를 더 많이 사용합니다.

 

마지막 버전의 Sentence 클래스에서 보듯이 제너레이터 표현식은 함수를 정의하고 호출할 필요없이 제너레이터를 생성하는 편리한 구문입니다. 반면 제너레이터 함수는 유연성이 더 높습니다. 여러 문장으로 구성된 복잡한 로직을 구현할 수 있고, 심지어 코루틴(coroutines)으로 사용할 수도 있습니다.

 

로직이 간단한 경우에는 제너레이터 표현식으로도 충분하며, Vector 클래스처럼 한 눈에 보기에도 더 쉽습니다.

 

이 두 가지 방식 중에 어떤 것을 선택할 것인가에 대한 저자의 규칙은 간단합니다. 제너레이터 표현식이 여러 줄에 걸쳐 있을 때는 가독성을 위해 제너레이터 함수를 사용합니다. 게다가 제너레이터 함수는 이름을 가지고 있으므로 재사용할 수도 있습니다.

 

지금까지 본 Sentence 예제는 전통적인 이터레이터의 역할을 하는 제너레이터의 예제입니다. 즉 컬렉션에서 항목들을 꺼내오는 역할을 합니다. 그러나 제너레이터는 데이터 출처에 무관하게 값을 생성하기 위해 사용할 수도 있습니다. 아래에서는 이런 사례들에 대해 살펴보겠습니다.

# 8. 또 다른 예제: 등차수열 제너레이터

전통적인 이터레이터 패턴은 모두 데이터 구조체를 뒤져서 항목들을 나열하기 위한 것입니다. 그러나 수열에서 다음 항목을 가져오는 메소드에 기반한 표준 인터페이스는 컬렉션에서 항목을 가져오는 대신 실행 도중에 항목을 생성하는 경우에도 유용하게 사용할 수 있습니다. 예를 들어 내장 함수 range()는 정수로 구성된 유한 등차수열을 생성하며, itertools.count() 함수는 무한 등차수열을 생성합니다.

 

특정 자료형의 숫자로 구성된 유한 등차수열을 생성하려면 어떻게 해야 할까요?

 

아래 예제 코드는 잠시 뒤에 살펴볼 ArithmeticProgression 클래스를 콘솔에서 테스트한 결과를 보여줍니다. 이 예제에서 사용한 생성자 시그니처는 ArithmeticProgression(begin, step[, end]) 입니다. range() 함수가 여기에서 사용한 ArithmeticProgression 클래스와 비슷하지만, range() 함수의 전체 시그니처는 range(start, stop[, step]) 입니다. 등차수열에서는 step이 필수고, end가 선택지이므로 여기서는 다른 시그니처를 사용했습니다. 아래 예제 코드는 테스트할 때마다 생성된 값들을 조사하기 위해 반환된 결과에 list() 생성자를 적용했습니다.

등차수열로 생성된 숫자들의 자료형이 파이썬 산술의 수치형 강제 변환 규칙에 의해 begin이나 step의 자료형을 따름에 주의합니다. 위 예제에서는 int, float, Fraction, Decimal 타입 숫자들의 리스트를 보여주고 있습니다

In [82]:
class ArithmeticProgression:
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end # None이면 무한수열
    
    def __iter__(self):
        # self.begin과 같은 값이 되지만, 이후에 더할 값에 맞춰 자료형을 강제 변환
        result = type(self.begin + self.step)(self.begin)
        forever = self.end is None
        index = 0
        while forever or result < self.end:
            yield result # 현재 result를 생성
            index += 1
            result = self.begin + self.step * index # 다음에 가져올 result를 미리 계산

###  8.1 itertools를 이용한 등차수열

(파이썬 3.9 기준) itertools 모듈에는 다양하고 재미있게 조합할 수 있는 19개의 제너레이터 함수가 있습니다.

 

예를 들어, itertools.count() 함수는 숫자를 생성하는 제너레이터를 반환합니다. 인수를 지젛아지 않으면 0에서 시작하는 수열을 생성합니다. 그러나 start와 stop 인수를 지정하면 앞에서 구현한 aritprog_gen() 함수와 아주 비슷한 결과를 낼 수 있습니다.

In [88]:
import itertools
gen = itertools.count(1, .5)
next(gen)

1

In [89]:
next(gen)

1.5

In [90]:
next(gen)

2.0

그러나 itertools.count()는 끝이 없습니다. 따라서 list(count())를 실행하면, 파이썬 인터프리터는 사용할 수 있는 메모리보다 더 큰 리스트를 만드려고 시도하면서 잠시 뒤 실패합니다.

 

그리고 itertools.takewhile() 이라는 함수도 있습니다. 이 함수는 다른 제너레이터를 소비하면서 주어진 조건식(predicate)이 False가 되면 중단되는 제너레이터를 생성합니다. 이 두 개의 제너레이터를 결합해서 다음과 같이 구현할 수 있습니다.

# 9,  표준 라이브러리 제너레이터 함수

필터링 제너레이터 함수

| Module | Function  |Description|
|----------|----------|----------|
|itertools|	ompress(it, selector_it)|두 개의 반복형을 병렬로 소비한다. selector_it의 해당 항목이 참일 때마다 it에서 항목을 생성한다.
|itertools|	dropwhile(predicate, it)|	predicate가 True인 값들을 스킵하다가, 처음 False인 값부터 추가 검사없이 남아 있는 항목들을 생성|
|(built-in)|	filter(predicate, it)|	predicate를 it의 각 항목에 적용해서 predicate(it)이 True라면 각 항목을 생성. predicate가 None이면 True인 값들을 모두 생성|
|itertools|	filterfalse(predicate, it)	|filter()와 같지만 반대의 로직을 적용. predicate로 False인 값들을 모두 생성한다.|
|itertools|	islice(it, stop) or islice(it, start, stop, step=1)	|s[:stop]이나 s[start:stop:step]과 비슷하게, 반복할 수 있는 모든 객체에 lazy 연산을 적용해서 it의 슬라이스 항목을 생성|
|itertools|	takewhile(predicate, it)|	predicate가 True로 계산되는 동안 모든 항목을 생성하고, False를 만나면 멈춘다.|

매핑 제너레이터 함수

| Module | Function  |Description|
|----------|----------|----------|
|itertools|	accumulate(it, [func])	|누적된 합계를 구한다. func을 제공하면, 처음 두 개의 항목에 func을 적용한 결과를 첫 번째 값으로 생성하며 it을 반복한다.|
|(built-in)|	enumerate(it, start=0)	|(인덱스, 항목) 형태의 튜플을 생성한다. 인덱스는 start부터 시작한다.|
|(built-in)	|map(func, it1, [it2, ..., itN])|	func을 각 it에 적용해서 결과를 생성한다. N개의 반복형이 주어지는 경우, func은 N개의 인수를 받아야 하며, N개의 반복형을 병렬로 소모한다.|
|itertools|	startmap(func, it)|	it의 각 항목에 func을 적용해서 결과를 생성한다. 입력된 it는 iit을 생성하고, func은 func(*iit) 형태로 호출된다.|

머지 제너레이터 함수 

| Module | Function  |Description|
|----------|----------|----------|
|itertools|	chain(it1, ..., itN)|	it1의 모든 항목을 생성한 후, 나머지 반복형을의 항목을 차례대로 생성|
|itertools|	chain.from_iterable(it)	|it에서 생성된 반복형 객체의 모든 항목을 생성한다. it이 생성한 항목은 반복할 수 있어야 한다 (ex, 반복형의 리스트)|
|itertools|	product(it1, ..., itN, repeat=1)|	데카르트 곱을 계산한다. 각 it의 항목을 이용해서 중첩된 for 루프가 생성하듯이 N-튜플을 생성한다. repeat는 it이 두 번 이상 소비되도록 허용한다.|
|(built-in)|	zip(it1, ..., itN)	|각 it의 항목을 병렬로 소비해서 N-튜플을 생성한다. 어느 하나의 it이 소모되면 중단한다.|
|itertools|	zip_longest(it1, ..., itN, fillvalue=None)|	각 it의 항목을 병렬로 소비해서 N-튜플을 생성한다. 가장 긴 it 기준으로 항목을 소모하며, 빈 값들은 fillvalue로 채워가며 생성한다.|

입력된 항목 하나를 여러개로 확장하는 제너레이터 함수

| Module | Function  |Description|
|----------|----------|----------|
|itertools|	combinations(it, out_len)|	it으로 생성된 항목에서 out_len개의 조합을 생성 
|itertools|	combinations_with_replacement(it, out_len)|	반복된 항목들의 포함을 포함해서, it로 생성된 항목에서 out_len개의 조합을 생성
|itertools|	count(start=0, step=1)	|start에서 시작해서 step만큼 증가시키며 숫자를 무한히 생성
|itertools|	cycle(it)|	각 항목의 사본을 저장한 후, 항목을 무한히 반복
|itertools|	permutations(it, out_len=None)|	it으로 생성된 항목에서 out_len개의 항목의 조합을 생성. 기본적으로 out_len은 len(list(it))이다.
|itertools|	repeat(item, [times]	|times를 지정하면 times만큼, 아니면 주어진 item을 무한히 반복해서 생성


재배치 제너레이터 함수

| Module | Function  |Description|
|----------|----------|----------|
|itertools|	groupby(it, key=None)|	(<key>, <group>)의 튜플을 생성한다. 이때 key는 그룹화 기준, group은 그룹 안의 항목을 생성하는 제너레이터이다.|
|(built-in)|	reversed(seq)|	seq안의 항목을 뒤에서부터 역순으로 생성한다. seq는 시퀀스이거나 __reversed__ 스페셜 메소드를 구현해야 한다.|
|itertools|	tee(it, n=2)	|n개의 제너레이터로 구성된 튜플을 하나 생성한다. 각 제너레이터는 입력된 it을 독립적으로 생성한다.|

# 10. 파이썬 3.3의 새로운 구문 : yield from 

In [99]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i
            
s = 'ABC'
t = tuple(range(3))
list(chain(s,t))

['A', 'B', 'C', 0, 1, 2]

In [104]:
def chain(*iterables):
    for i in iterables:
        yield from i
            
list(chain(s,t))

['A', 'B', 'C', 0, 1, 2]

# 11. 반복형을 리듀스하는 함수
  
  
  아래의 표에 나열된 함수들은 모두 반복형을 입력받아 하나의 값을 반환합니다. 이 함수는 흔히 'reducing', 'folding', 또는 'accumulating' 함수라고 합니다. 사실 여기에 나열된 함수는 모두 functools.reduce() 함수로 구현할 수 있지만, 자주 발생하는 특정 문제를 쉽게 처리하기 때문에 별도의 내장형 함수로 존재합니다. 그리고 all()과 any()는 단락 평가(short-circuit evaluation) 함수로서, reduce()로 최적화할 수 없습니다. 단락 평가를 하는 경우에는 결과가 확정되는 즉시 이터레이터 소비를 중단합니다.

| Module | Function  |Description|
|----------|----------|----------|
|(built-in)|	all(it)|	it의 모든 항목이 True면 True를, 아니면 False를 반환.all([])은 True를 반환한다.|
|(built-in)|	any(it)	|it의 항목들 중 하나라도 True라면 True를, 아니면 False를 반환.any([])는 False를 반환한다.|
|(built-in)|	max(it, [key=,] [default=])	|it의 항목들 중 최댓값을 반환한다. key는 sorted()에서 사용하는 정렬 함수와 동일한 함수며, it이 비어 있을 때는 default로 반환한다. max(arg1, arg2,..., [key=]) 형태로도 호출 가능. 이때는 인수들 중 최댓값이 반환된다.|
|(built-in)|min(it, [key=,] [default=])|	max와 동일하면 최솟값을 반환한다.|
|functools|	reduce(func, it, [initial])|	처움 두 개의 항목에 func을 적용하고, 그 결과와 세 번째 항목에 또 func을 적용하는 과정을 반복한 결과를 반환한다. initial이 주어지면 initial과 첫 항목에 func을 적용하면서 시작한다.|
|(built-in)|sum(it, start=0)|	it 항목의 합계에 선택적인 start 값을 더한 값을 반환한다. 실수형의 경우 math.fsum()을 사용하면 정밀도가 향상된다.|

# 12. iter() 함수 들여다 보기

금까지 본 것처럼 파이썬은 어떤 객체 x를 반복해야 할 때 iter(x)를 호출합니다.

 

그러나 이 함수는 일반 함수나 콜러블 객체로부터 이터레이터를 생성하기 위해 두 개의 인수를 전달해서 호출할 수도 있습니다. 이렇게 사용하려면, 첫 번째 인수는 값을 생성하기 위해 인수없이 반복적으로 호출되는 콜러블이어야 하며, 두 번째 인수는 구분 표시(sentinel)로서, 콜러블에서 이 값이 반환되면 이터레이터가 StopIteration 예외를 발생시키도록 만듭니다.


다음 예제는 1이 나올 때까지 육면체 주사위를 굴리기 위해 iter() 함수를 사용하는 방법을 보여줍니다.

In [115]:
from random import randint

def d6():
    return randint(1,6)

d6_iter = iter(d6, 1)
d6_iter

In [116]:
for roll in d6_iter:
    print(roll)

5
3
4
2
2
2
2
4
2


여기서 iter() 함수가 callable_iterator 객체를 반환함에 주의해야 합니다. 예제 안에 있는 for 루프는 아무리 오래 실행하더라도 결고 1을 출력하지 않습니다. 1이 구분 표시이기 때문입니다. 이터레이터와 마찬가지로 d6_iter 객체는 일단 소모하고 난 후에는 쓸모가 없어집니다. 다시 시작하려면 iter() 함수를 한 번 더 호출해서 이터레이터를 다시 만들어야 합니다.